# Few-Shot Training

In [2]:
import os
import re
import pandas as pd
from datetime import date
from setfit import SetFitModel
from setfit import sample_dataset
from datasets import Dataset
from setfit import TrainingArguments
from setfit import Trainer
today = date.today().strftime('%Y-%m-%d')

c:\Users\gbernar\OneDrive - MORNINGSTAR INC\HF\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#Multilingual model
model = SetFitModel.from_pretrained("BAAI/bge-m3")

In [3]:
# English smaller model
model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [49]:
files = os.listdir(os.getcwd()+"\\Fewshot_Labelled")
files

['Custodian_Fee-2024-06-18-df_FEWSHOT.csv',
 'Custodian_Fee-2024-06-18-df_FEWSHOT.xlsx',
 'Fund_Investor-2024-06-18-df6_FEWSHOT.csv',
 'Fund_Investor-2024-06-18-df6_FEWSHOT.xlsx',
 'Investment_Criteria-2024-06-18-df_FEWSHOT.csv',
 'Investment_Criteria-2024-06-18-df_FEWSHOT.xlsx']

In [50]:
def get_most_recent_file(folder_path):
    # Get a list of files in the folder
    files = os.listdir(folder_path)
    
    # Full paths of the files
    full_paths = [os.path.join(folder_path, file) for file in files]
    
    # Filter out directories, leaving only files
    full_paths = [path for path in full_paths if os.path.isfile(path)]
    
    # Get the most recent file by comparing modification times
    most_recent_file = max(full_paths, key=os.path.getmtime)
    return most_recent_file

In [51]:
file_path = get_most_recent_file(os.getcwd()+"\\Fewshot_Labelled")

In [53]:
file_path

'C:\\Users\\gbernar\\OneDrive - MORNINGSTAR INC\\Aristotle_SetFit\\Fewshot_Labelled\\Investment_Criteria-2024-06-18-df_FEWSHOT.csv'

In [54]:
df=pd.read_csv(file_path)

In [55]:
# Step 2: Convert the pandas DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [58]:
# Sample the dataset
train_dataset = sample_dataset(dataset, label_column="label", num_samples=130)
model.labels = [0, 1]

C:\Users\gbernar\OneDrive - MORNINGSTAR INC\VENV\Lib\site-packages\setfit\data.py:154: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.apply(lambda x: x.sample(min(num_samples, len(x)), random_state=seed))


In [42]:
args = TrainingArguments(
    batch_size=32,
    num_epochs=1,
) 
args.eval_strategy = args.evaluation_strategy #add update of args for evaluate as eval_strategy was deprecated.

In [43]:
# Activate Trainer

In [44]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)

Map:   0%|          | 0/159 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
model.save_pretrained(f"Description_Selection")